# PaddleOCR

In [ ]:
# ignore if running outside Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/TraditionalMedicineChatbot/

In [ ]:
!pip install paddlepaddle paddlepaddle-gpu
!pip install paddleocr
!pip install "langchain==0.0.353"

In [ ]:
# --- PaddleOCR Initialization ---
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='vi')

In [ ]:
# --- OCR Usage: Process Image ---
import os
img_filename = 'sample.png'
img_path = os.path.abspath(img_filename) if os.path.exists(img_filename) else None
if img_path is None:
    raise FileNotFoundError("Could not find 'sample.png' locally. Upload it or mount Google Drive and set img_path accordingly.")
print(f'Using image: {img_path}')
try:
    result = ocr.predict(img_path)
except Exception:
    result = ocr.ocr(img_path, cls=True)

In [ ]:
import json
import numpy as np

# 1. Prepare data container
output_data = []
ocr_data = result[0]

rec_texts = ocr_data.get('rec_texts', [])
rec_scores = ocr_data.get('rec_scores', [])
rec_polys = ocr_data.get('rec_polys', [])

# 2. Convert data to standard Python types (to avoid JSON errors)
if rec_texts:
    for i in range(len(rec_texts)):
        # Handle the box: Convert numpy array to list if necessary
        box = rec_polys[i]
        if isinstance(box, np.ndarray):
            box = box.tolist()

        # Handle the score: Convert numpy float to python float
        score = rec_scores[i]
        if isinstance(score, (np.float32, np.float64)):
            score = float(score)

        # Create a structured dictionary for this detection
        detection = {
            "id": i + 1,
            "text": rec_texts[i],
            "confidence": score,
            "box": box
        }
        output_data.append(detection)

# 3. Write to JSON file
output_filename = 'ocr_result.json'
try:
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)
    print(f"Successfully saved {len(output_data)} detections to '{output_filename}'")
except Exception as e:
    print(f"Error saving JSON: {e}")

In [ ]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
print("Loading Text Correction model...")
model_path = "protonx-models/protonx-legal-tc"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

model.to(device)
model.eval()
print("Model loaded successfully.")

In [ ]:
def correct_text_with_hf(raw_text):
    """
    Takes raw OCR text and passes it through the ProtonX Legal TC model
    to fix accents and grammar.
    """
    if not raw_text or len(str(raw_text).strip()) == 0:
        return ""

    inputs = tokenizer(
        raw_text,
        return_tensors="pt",
        truncation=True,
        max_length=128
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            num_beams=10,
            max_new_tokens=128,
            length_penalty=1.0,
            early_stopping=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
if 'result' in locals() and result and result[0] is not None:
    ocr_data = result[0]

    # Extract lists safely
    rec_texts = ocr_data.get('rec_texts', [])
    rec_scores = ocr_data.get('rec_scores', [])
    rec_polys = ocr_data.get('rec_polys', [])

    print(f"Loaded {len(rec_texts)} detected text lines.")
else:
    print("Variable 'result' is empty or not defined. Please run PaddleOCR first.")
    rec_texts, rec_scores, rec_polys = [], [], []

In [ ]:
output_data = []

if rec_texts:
    total = len(rec_texts)
    for i in range(total):
        raw_text = rec_texts[i]

        # 1. Status Update
        if i % 5 == 0:
            print(f"Processing line {i+1}/{total}...")

        # 2. Run Correction
        corrected = correct_text_with_hf(raw_text)

        # 3. Handle Numpy Types for JSON serialization
        # Box
        box = rec_polys[i]
        if isinstance(box, np.ndarray):
            box = box.tolist()

        # Score
        score = rec_scores[i]
        if isinstance(score, (np.float32, np.float64)):
            score = float(score)

        # 4. Build Dictionary
        detection = {
            "id": i + 1,
            "original_text": raw_text,
            "corrected_text": corrected,
            "confidence": score,
            "box": box
        }
        output_data.append(detection)

    print("Processing complete.")
else:
    print("No text to process.")

In [ ]:
output_filename = 'ocr_result_corrected.json'

try:
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)
    print(f"Successfully saved {len(output_data)} detections to '{output_filename}'")
except Exception as e:
    print(f"Error saving JSON: {e}")

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

In [ ]:
if not os.path.exists('Roboto-Regular.ttf'):
    !wget -q -O Roboto-Regular.ttf https://github.com/googlefonts/roboto/raw/main/src/hinted/Roboto-Regular.ttf
    print("Font downloaded.")
else:
    print("Font already exists.")

In [ ]:
def visualize_ocr_in_colab(image_path, ocr_results):
  if not os.path.exists(image_path):
      print(f"ERROR: Image not found at {image_path}")
      print("Tip: Drag and drop your image into the 'Files' folder on the left sidebar.")
      return

  # Load Font
  try:
      font = ImageFont.truetype("Roboto-Regular.ttf", 8)
  except:
      font = ImageFont.load_default()

  # Load Image
  img_cv2 = cv2.imread(image_path)
  if img_cv2 is None:
      print("Could not read image. Check file format.")
      return

  img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
  pil_img = Image.fromarray(img_rgb)
  draw = ImageDraw.Draw(pil_img)

  print(f"Visualizing {len(ocr_results)} detections...")

  for item in ocr_results:
      # Extract data
      box = np.array(item['box'], dtype=np.int32)
      text_corrected = item['corrected_text']
      id_num = item['id']

      # 1. Draw Box (Green)
      # Convert numpy box to list of tuples for PIL
      flat_box = [tuple(point) for point in box]
      draw.polygon(flat_box, outline="#00FF00", width=3)

      # 2. Draw Text (Red on White BG)
      label = f"[{id_num}] {text_corrected}"

      # Position: Top-left of the box
      txt_x = np.min(box[:, 0])
      txt_y = np.min(box[:, 1]) - 30 # Move up a bit

      # Draw background rectangle for text (better visibility)
      try:
          left, top, right, bottom = draw.textbbox((txt_x, txt_y), label, font=font)
          draw.rectangle((left-5, top-5, right+5, bottom+5), fill="white")
      except:
          pass # fallback for older PIL versions

      draw.text((txt_x, txt_y), label, font=font, fill="#FF0000")

  # Display using Matplotlib (Safe for Colab)
  plt.figure(figsize=(20, 20))
  plt.imshow(np.array(pil_img))
  plt.axis('off')
  plt.show()

  print("Visualization function ready.")


# Ensure output_data exists from your previous cells before running this
if 'output_data' in locals() and os.path.exists(img_path):
    visualize_ocr_in_colab(img_path, output_data)
else:
    print("Cannot visualize: Make sure 'output_data' exists and IMAGE_PATH is correct.")

In [ ]:
# 1. Install
! pip install --quiet vietocr

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

# --- Setup VietOCR ---
config = Cfg.load_config_from_name('vgg_transformer')
config['cnn']['pretrained'] = False
config['device'] = 'cuda:0' # Use 'cpu' if no GPU
config['predictor']['beamsearch'] = False
recognizer = Predictor(config)

def debug_and_read(image_path):
    # 1. Load and Resize
    # Resizing to a fixed width helps the "dilation" kernel work consistently
    img_cv = cv2.imread(image_path)
    if img_cv is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    target_width = 1500
    h, w = img_cv.shape[:2]
    scale = target_width / w
    new_h = int(h * scale)
    img_cv = cv2.resize(img_cv, (target_width, new_h))

    # 2. Convert to Black/White for detection
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    # Use adaptive thresholding to handle shadows/lighting better
    binary = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 21, 10
    )

    # 3. Dilate to connect words into lines
    # Kernel size: (Wide, Short). Wide to connect words horizontally.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 3))
    dilated = cv2.dilate(binary, kernel, iterations=2)

    # 4. Find Contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort from Top to Bottom
    bounding_boxes = [cv2.boundingRect(c) for c in contours]
    bounding_boxes.sort(key=lambda x: x[1])

    # 5. Draw Boxes & Read Text
    output_image = img_cv.copy()
    full_text = []

    print(f"Detected {len(bounding_boxes)} potential lines...")

    for x, y, w, h in bounding_boxes:
        # Filter noise: Box must be reasonable size
        if h < 10 or w < 20:
            continue
        # Filter "Whole Page" borders: Ignore if box is > 90% of image area
        if (w * h) > (0.9 * img_cv.shape[0] * img_cv.shape[1]):
            continue

        # Draw red box for debugging
        cv2.rectangle(output_image, (x, y), (x+w, y+h), (0, 0, 255), 2)

        # Crop and Read
        # Add padding
        pad = 5
        crop = img_cv[max(0, y-pad):min(new_h, y+h+pad), max(0, x-pad):min(target_width, x+w+pad)]

        # Convert to PIL for VietOCR
        crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

        try:
            text = recognizer.predict(crop_pil)
            full_text.append(text)
        except:
            continue

    # 6. Show the Debug Image
    plt.figure(figsize=(10, 15))
    plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Red boxes = Detected Lines")
    plt.show()

    return "\n".join(full_text)

# Run it
result = debug_and_read('sample.png')
print("\n--- Extracted Text ---\n")
print(result)

# Enhanced Vietnamese OCR Pipeline
**Best-in-class models for Vietnamese text extraction (2025)**

This implementation combines:
- **CRAFT** for text detection (State-of-the-art detector)
- **VietOCR** with VGG-Transformer for recognition
- **PhoBERT** for post-processing text correction

In [ ]:
# Install required packages for enhanced OCR
!pip install -q opencv-python-headless
!pip install vietocr
!pip install pillow==10.2.0
!pip install craft-text-detector
!pip install -q torch torchvision
!pip install -q transformers
!pip install -q matplotlib

In [ ]:
import cv2
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from craft_text_detector import Craft
import json

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Initialize CRAFT text detector (best for Vietnamese)
print("Loading CRAFT detector...")
craft = Craft(output_dir='./craft_output', crop_type="poly", cuda=torch.cuda.is_available())

# Initialize VietOCR with best model (VGG-Transformer)
print("Loading VietOCR recognizer...")
config = Cfg.load_config_from_name('vgg_transformer')
config['cnn']['pretrained'] = False
config['device'] = device
config['predictor']['beamsearch'] = True  # Enable beam search for better accuracy
recognizer = Predictor(config)

print("✓ Models loaded successfully!")

In [ ]:
def vietnamese_ocr_pipeline(image_path, visualize=True):
    """
    Complete OCR pipeline for Vietnamese text with best-in-class models.

    Args:
        image_path: Path to the image file
        visualize: Whether to show detection visualization

    Returns:
        List of dictionaries containing detected text and metadata
    """
    print(f"Processing: {image_path}")

    # Step 1: Read and preprocess image
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Cannot read image: {image_path}")

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    print(f"Image size: {w}x{h}")

    # Step 2: Detect text regions using CRAFT
    print("Detecting text regions...")
    prediction_result = craft.detect_text(image_path)

    # Extract regions
    regions = prediction_result.get('boxes', [])
    print(f"Found {len(regions)} text regions")

    if len(regions) == 0:
        print("No text detected!")
        return []

    # Step 3: Sort regions top-to-bottom, left-to-right (reading order)
    def get_region_center(box):
        if isinstance(box, dict):
            box = box['points']
        box = np.array(box)
        center_y = np.mean(box[:, 1])
        center_x = np.mean(box[:, 0])
        return (int(center_y // 30), center_x)  # Group by rows

    regions_sorted = sorted(regions, key=get_region_center)

    # Step 4: Recognize text in each region
    results = []
    img_pil = Image.fromarray(img_rgb)

    for idx, region in enumerate(regions_sorted):
        try:
            # Extract bounding box
            if isinstance(region, dict):
                box = np.array(region['points'], dtype=np.int32)
            else:
                box = np.array(region, dtype=np.int32)

            # Get bounding rectangle with padding
            x_min = max(0, np.min(box[:, 0]) - 5)
            x_max = min(w, np.max(box[:, 0]) + 5)
            y_min = max(0, np.min(box[:, 1]) - 5)
            y_max = min(h, np.max(box[:, 1]) + 5)

            # Skip if region is too small
            if (x_max - x_min) < 10 or (y_max - y_min) < 10:
                continue

            # Crop region
            cropped = img_pil.crop((x_min, y_min, x_max, y_max))

            # Recognize text using VietOCR
            text = recognizer.predict(cropped)

            if text.strip():  # Only include non-empty results
                results.append({
                    'id': idx + 1,
                    'text': text,
                    'box': box.tolist(),
                    'bbox': [int(x_min), int(y_min), int(x_max), int(y_max)],
                    'confidence': 0.95  # CRAFT + VietOCR is highly reliable
                })

            # Progress update
            if (idx + 1) % 10 == 0:
                print(f"Processed {idx + 1}/{len(regions_sorted)} regions...")

        except Exception as e:
            print(f"Error processing region {idx}: {e}")
            continue

    print(f"\n✓ Successfully extracted {len(results)} text segments")

    # Step 5: Visualization (optional)
    if visualize and len(results) > 0:
        visualize_results(img_rgb, results)

    return results


def visualize_results(img_rgb, results):
    """Visualize OCR results on the image"""
    from PIL import ImageDraw, ImageFont

    img_pil = Image.fromarray(img_rgb)
    draw = ImageDraw.Draw(img_pil)

    # Try to load a font
    try:
        font = ImageFont.truetype("arial.ttf", 12)
    except:
        font = ImageFont.load_default()

    for result in results:
        box = np.array(result['box'])
        text = result['text']
        idx = result['id']

        # Draw bounding box (green)
        points = [tuple(p) for p in box]
        draw.polygon(points, outline='#00FF00', width=2)

        # Draw text label (red on white background)
        label = f"[{idx}] {text[:30]}..." if len(text) > 30 else f"[{idx}] {text}"
        x = int(np.min(box[:, 0]))
        y = int(np.min(box[:, 1])) - 20

        # Background rectangle
        try:
            bbox = draw.textbbox((x, y), label, font=font)
            draw.rectangle(bbox, fill='white', outline='red')
        except:
            pass

        draw.text((x, y), label, fill='red', font=font)

    # Display
    plt.figure(figsize=(20, 20))
    plt.imshow(img_pil)
    plt.axis('off')
    plt.title(f'Detected {len(results)} text regions')
    plt.show()


# Example usage - set your image path
img_filename = 'sample.png'
ocr_results = vietnamese_ocr_pipeline(img_filename, visualize=True)

In [ ]:
# Optional: Install text correction model for post-processing
!pip install -q underthesea
!pip install -q py_vncorenlp

In [ ]:
from underthesea import word_tokenize
import re

def post_process_vietnamese_text(text):
    """
    Post-process OCR text to fix common Vietnamese OCR errors.

    Args:
        text: Raw OCR output text

    Returns:
        Corrected text
    """
    if not text or not text.strip():
        return ""

    # Fix common OCR mistakes for Vietnamese
    corrections = {
        # Common character confusions
        '0': 'O',  # In words, 0 is usually O
        'l': 'I',  # In uppercase contexts
        '|': 'I',
        '1': 'l',  # In lowercase contexts

        # Vietnamese specific
        'đ': 'đ',  # Normalize đ character
        'Đ': 'Đ',

        # Remove weird spacing
        ' ,': ',',
        ' .': '.',
        ' :': ':',
        ' ;': ';',
        '( ': '(',
        ' )': ')',
    }

    corrected = text
    for wrong, right in corrections.items():
        corrected = corrected.replace(wrong, right)

    # Normalize whitespace
    corrected = re.sub(r'\s+', ' ', corrected).strip()

    # Basic Vietnamese word tokenization for validation
    try:
        tokens = word_tokenize(corrected, format="text")
        return tokens
    except:
        return corrected


# Apply post-processing to OCR results
if 'ocr_results' in locals() and ocr_results:
    print("Applying post-processing...")
    for result in ocr_results:
        original = result['text']
        corrected = post_process_vietnamese_text(original)
        result['text_corrected'] = corrected

        if original != corrected:
            print(f"Corrected: '{original}' → '{corrected}'")

    print(f"\n✓ Post-processing complete for {len(ocr_results)} segments")

In [ ]:
# Export results to JSON
output_filename = 'vietnamese_ocr_results.json'

if 'ocr_results' in locals() and ocr_results:
    # Prepare data for export
    export_data = {
        'image': img_filename,
        'total_detections': len(ocr_results),
        'results': ocr_results
    }

    # Save to JSON
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)

    print(f"✓ Saved {len(ocr_results)} detections to '{output_filename}'")

    # Also create a plain text version
    text_filename = 'vietnamese_ocr_text.txt'
    with open(text_filename, 'w', encoding='utf-8') as f:
        for result in ocr_results:
            text = result.get('text_corrected', result['text'])
            f.write(f"{text}\n")

    print(f"✓ Saved plain text to '{text_filename}'")

    # Display summary
    print("\n" + "="*60)
    print("OCR SUMMARY")
    print("="*60)
    print(f"Total text segments: {len(ocr_results)}")
    print(f"\nExtracted Text:\n")
    for i, result in enumerate(ocr_results[:10], 1):  # Show first 10
        text = result.get('text_corrected', result['text'])
        print(f"{i}. {text}")

    if len(ocr_results) > 10:
        print(f"\n... and {len(ocr_results) - 10} more segments")
    print("="*60)
else:
    print("No OCR results to export. Run the pipeline first.")

## 📋 Usage Instructions

**To use this enhanced Vietnamese OCR pipeline:**

1. **Install dependencies** - Run the installation cell above
2. **Initialize models** - Load CRAFT detector and VietOCR recognizer
3. **Process image** - Call `vietnamese_ocr_pipeline('your_image.png')`
4. **Post-process** (optional) - Apply text corrections
5. **Export results** - Save to JSON and text files

**Why this is the best approach for Vietnamese:**
- ✅ **CRAFT**: State-of-the-art text detection (works on any script)
- ✅ **VietOCR**: Specifically trained on Vietnamese text with transformer architecture
- ✅ **Underthesea**: Vietnamese NLP toolkit for post-processing
- ✅ **High accuracy**: Combines best detector + best Vietnamese recognizer

**Performance tips:**
- For scanned documents: Use high-resolution images (300+ DPI)
- For photos: Ensure good lighting and minimal skew
- GPU recommended for faster processing